## Here, we begin by installing the required libraries.

In [3]:
!pip install roboflow
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.1/74.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 7.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [1]:
import torch
from IPython.display import display
from roboflow import Roboflow
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

In [5]:
rf = Roboflow(api_key="lRIgGm3WjzsZyY4OcgnE")
project = rf.workspace("chinh").project("person_camera_security1")
version = project.version(20)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.2, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to person_camera_security1-20 in yolov8:: 100%|██████████| 6328/6328 [00:01<00:00, 4386.92it/s]


###We do transfer learning on the pretrained general purpose YOLOv8 model nano. This model can detect 80 different classes, the person class is the interest to us. Our aim is to improve its accuracy in detecting persons. So, we use the dataset from ROBOFLOW that is specifically data from webcams in order to do this transfer learning.

###Note that as per the YOLOv8 documentation, data augmentation is being done inside the library by default, as well as other measures to prevent overfitting. It also uses adam optimizer in updating the weights.

### Also note that this training is done using the local GPU resources, and it is done on the NANO model because it is the smallest one and thus the fastest. This is important in order to be able to export the trained model later into the project code which will run on Raspberry pi.

In [ ]:

# Load a model
model = YOLO("yolov8n.pt")

# Use the model
model.train(data="/content/person_camera_security1-20/data.yaml", epochs=100)  # train the model
metrics = model.val()  # evaluate model performance on the validation set
#results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
path = model.export(format="onnx")  # export the model to ONNX format

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/person_camera_security1-20/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, s

100%|██████████| 755k/755k [00:00<00:00, 17.9MB/s]


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /content/person_camera_security1-20/train/labels... 2500 images, 42 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:01<00:00, 1613.50it/s]


train: New cache created: /content/person_camera_security1-20/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

val: Scanning /content/person_camera_security1-20/valid/labels.cache... 436 images, 3 backgrounds, 0 corrupt: 100%|██████████| 436/436 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:08<00:00,  3.14it/s]


                   all        436        989      0.815      0.523      0.575      0.392
                 chair        436         22      0.974      0.364      0.527      0.408
               handbag        436         18      0.826       0.79      0.774      0.583
                laptop        436          4      0.913       0.75      0.888      0.598
                person        436        929      0.946      0.897      0.962      0.706
          refrigerator        436          4      0.434      0.217      0.207      0.148
            teddy bear        436          3      0.957      0.667      0.685      0.208
                    tv        436          6      0.473        0.5      0.561      0.483
                  vase        436          3          1          0          0          0
Speed: 0.2ms preprocess, 6.1ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/train22



100%|██████████| 476k/476k [00:00<00:00, 12.8MB/s]

image 1/1 /content/bus.jpg: 640x480 1 chair, 4 persons, 17.4ms
Speed: 2.4ms preprocess, 17.4ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 480)


Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.30GHz)

PyTorch: starting from 'runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 14, 8400) (6.0 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 93.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 10.0s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 10.9s, saved as 'runs/detect/train2/weights/best.onnx' (11.7 MB)

Export complete (12.6s)
Results saved to /content/runs/detect/train2/weights
Predict:         yolo predict task=detect model=runs/detect/train2/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train2/weights/best.onnx imgsz=640 data=/content/person_camera_security1

In [ ]:
ourmodel = model


validate = ourmodel.val()

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/person_camera_security1-20/valid/labels.cache... 436 images, 3 backgrounds, 0 corrupt: 100%|██████████| 436/436 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:09<00:00,  3.01it/s]


                   all        436        989      0.815      0.523      0.575      0.392
                 chair        436         22      0.974      0.364      0.527      0.408
               handbag        436         18      0.826       0.79      0.774      0.583
                laptop        436          4      0.913       0.75      0.888      0.598
                person        436        929      0.946      0.897      0.962      0.706
          refrigerator        436          4      0.434      0.217      0.207      0.148
            teddy bear        436          3      0.957      0.667      0.685      0.208
                    tv        436          6      0.473        0.5      0.561      0.483
                  vase        436          3          1          0          0          0
Speed: 0.6ms preprocess, 4.1ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train23


###as we can see above, our model is doing the best with the person on the validation data. This is indeed what we want.

In [6]:
default_model = YOLO("yolov8n.pt")
my_model = YOLO("best.pt")



In [9]:
resultsmine= my_model.predict("/content/person_camera_security1-20/test/images")



image 1/222 /content/person_camera_security1-20/test/images/000000009400_jpg.rf.6b24df2c7ea9237bef02155db66210be.jpg: 480x640 1 person, 559.0ms
image 2/222 /content/person_camera_security1-20/test/images/000000017379_jpg.rf.da1abc9974f8577a50c3b2d6ff35a890.jpg: 640x480 1 tv, 173.9ms
image 3/222 /content/person_camera_security1-20/test/images/000000031248_jpg.rf.f851bb802826329f891d38f439279b99.jpg: 448x640 1 person, 168.4ms
image 4/222 /content/person_camera_security1-20/test/images/000000041872_jpg.rf.de3dd0e396b951017d0cf92859eee6c6.jpg: 448x640 (no detections), 163.7ms
image 5/222 /content/person_camera_security1-20/test/images/000000046031_jpg.rf.f19c45d22cb429ae261af070413a3ece.jpg: 480x640 1 laptop, 174.9ms
image 6/222 /content/person_camera_security1-20/test/images/000000057232_jpg.rf.eae866119ba8b0526b7d479ca62f8f7d.jpg: 640x640 (no detections), 226.2ms
image 7/222 /content/person_camera_security1-20/test/images/000000063740_jpg.rf.978117b93dc39ce1e37daeeb256478b6.jpg: 480x640

In [10]:
resultsdefault=  default_model.predict("/content/person_camera_security1-20/test/images")



image 1/222 /content/person_camera_security1-20/test/images/000000009400_jpg.rf.6b24df2c7ea9237bef02155db66210be.jpg: 480x640 9 persons, 4 cups, 6 laptops, 1 mouse, 1 keyboard, 189.1ms
image 2/222 /content/person_camera_security1-20/test/images/000000017379_jpg.rf.da1abc9974f8577a50c3b2d6ff35a890.jpg: 640x480 1 tv, 2 sinks, 167.8ms
image 3/222 /content/person_camera_security1-20/test/images/000000031248_jpg.rf.f851bb802826329f891d38f439279b99.jpg: 448x640 2 chairs, 1 couch, 2 potted plants, 2 books, 147.9ms
image 4/222 /content/person_camera_security1-20/test/images/000000041872_jpg.rf.de3dd0e396b951017d0cf92859eee6c6.jpg: 448x640 2 chairs, 1 couch, 1 bed, 1 tv, 1 clock, 147.4ms
image 5/222 /content/person_camera_security1-20/test/images/000000046031_jpg.rf.f19c45d22cb429ae261af070413a3ece.jpg: 480x640 1 tv, 1 laptop, 2 mouses, 1 keyboard, 2 cell phones, 165.4ms
image 6/222 /content/person_camera_security1-20/test/images/000000057232_jpg.rf.eae866119ba8b0526b7d479ca62f8f7d.jpg: 640x64

ValueError: Found input variables with inconsistent numbers of samples: [484, 492]